In [1]:
from meditations_rag.services.loader import DocumentLoaderService
from meditations_rag.pipelines.ingest import IngestPipeline

2025-11-18 00:04:13 | INFO     | Logging initialized for meditations-rag v0.1.0
2025-11-18 00:04:13 | INFO     | Environment: development


In [2]:
soure_file = "C:\\Users\\mrudh\\Documents\\Projects\\ProfileProject\\Agentic-RAG\\meditations-rag\\data\\Marcus-Aurelius-Meditations.pdf"

In [3]:
from llama_index.readers.file import PyMuPDFReader

reader = PyMuPDFReader()
loader = DocumentLoaderService(reader=reader)
ingest_pipeline = IngestPipeline(loader=loader)

In [4]:
import nest_asyncio
nest_asyncio.apply()

documents = await ingest_pipeline.ingest(file_path=soure_file)

2025-11-18 00:04:15 | INFO     | Starting ingestion for file: C:\Users\mrudh\Documents\Projects\ProfileProject\Agentic-RAG\meditations-rag\data\Marcus-Aurelius-Meditations.pdf
2025-11-18 00:04:15 | INFO     | Completed ingestion for file: C:\Users\mrudh\Documents\Projects\ProfileProject\Agentic-RAG\meditations-rag\data\Marcus-Aurelius-Meditations.pdf in 0.66 seconds
2025-11-18 00:04:15 | INFO     | Total documents ingested: 128


In [5]:
documents[0]

Document(id_='6948871f-fbe0-4f2c-a493-a5de4aa0e66c', embedding=None, metadata={'total_pages': 128, 'file_path': 'C:\\Users\\mrudh\\Documents\\Projects\\ProfileProject\\Agentic-RAG\\meditations-rag\\data\\Marcus-Aurelius-Meditations.pdf', 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=" \nMarcus Aurelius' Meditations - tr. Casaubon v. 8.16, uploaded to www.philaletheians.co.uk, 14 July 2013 \nPage 1 of 128 \nThe meditations of \nMarcus Aurelius Antoninus \nOriginally translated by Meric Casaubon \n \nAbout this edition \nMarcus Aurelius Antoninus Augustus was Emperor of Rome from 161 to his death, \nthe last of the “Five Good Emperors.” He was nephew, son-in-law, and adoptive son \nof Antonius Pius. Marcus Aurelius was one of the most important Stoic philosophers, \ncited by H.P. Blavatsky amongst famous classic sage